In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [15]:
landmarks = ['legs']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [16]:
with open('wrist.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [23]:
def export_landmark(results, wrist):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, wrist)

        with open('legs.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [24]:
def rescale_frame(frame, scale):    # works for image, video, live video
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA)

In [26]:
cap = cv2.VideoCapture("Data/squat/squat2.mp4")
cap.set(3, 1440)
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    #Loop video
    while cap.isOpened():
        ret, frame = cap.read()
        #frame_resized = rescale_frame(frame, scale=.6)
        
        #Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make detection
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        k = cv2.waitKey(1)

        if k == ord("w"):
            export_landmark(results, "legs_normal")
            cv2.putText(image, "UP", (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), cv2.LINE_AA)
        if k == ord("e"):
            export_landmark(results, "legs_wide")
            cv2.putText(image, "DOWN", (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), cv2.LINE_AA)
        if k == ord("r"):
            export_landmark(results, "legs_narrow")
            cv2.putText(image, "UP", (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), cv2.LINE_AA)
        #Display the annotated frame
        image_resized = rescale_frame(image, scale=.6)
        cv2.imshow("Mediapipe Feed", image_resized)

        #Break the loop
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
#Release the video capture object and close the display window  
cap.release()
cv2.destroyAllWindows()

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [15]:
with open('train.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [17]:
dfr = pd.read_csv('squat.csv')
dfr.to_csv('train.csv', mode='a', header=False, index=False)